In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from seleniumbase import SB

# URLs cần truy cập
PROFILE_URL = "https://fap.fpt.edu.vn/User/Profile.aspx"
ACAD_URL = "https://fap.fpt.edu.vn/App/AcadAppView.aspx"
CSV_FILE = "don_tu.csv"

# Lấy Student ID từ Profile page
def get_student_id_from_profile(driver):
    driver.get(PROFILE_URL)
    while True:
        try:
            if "Profile.aspx" in driver.current_url:
                break
            driver.title
            time.sleep(1)
        except:
            print("❌ Trình duyệt đã đóng khi vào trang Profile.")
            return None

    time.sleep(1)
    try:
        student_id = driver.find_element("id", "ctl00_mainContent_lblRollNumber").text.strip()
        print(f"🎓 Mã số sinh viên: {student_id}")
        return student_id
    except Exception as e:
        print(f"❌ Không lấy được Student ID: {e}")
        return None

def extract_acad_data_from_page_source(html, student_id):
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("div", id="ctl00_mainContent_content").find("table")
    rows = table.find_all("tr")[1:]  # Bỏ dòng tiêu đề

    data = []
    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 7:
            record = {
                "StudentID": student_id,
                "Type": cols[0].text.strip(),
                "Purpose": cols[1].text.strip(),
                "CreateDate": cols[2].text.strip(),
                "ProcessNote": cols[3].text.strip(),
                "File": cols[4].text.strip(),
                "Status": cols[5].text.strip(),
                "Timestamp": cols[6].text.strip()
            }
            data.append(record)

    return pd.DataFrame(data)

def write_new_csv(df, csv_file):
    df.to_csv(csv_file, index=False, encoding="utf-8-sig")
    print(f"✅ Đã lưu dữ liệu {len(df)} dòng vào: {csv_file}")

# Thực thi
with SB(uc=True, headless=False) as sb:
    driver = sb.driver
    driver.set_window_size(1920, 1080)

    print("🌐 Mở FAP và chờ bạn đăng nhập bằng tay...")
    driver.get("https://fap.fpt.edu.vn")

    # Đợi user đăng nhập
    while True:
        try:
            if "Student.aspx" in driver.current_url:
                break
            driver.title
            time.sleep(1)
        except:
            print("❌ Trình duyệt đã đóng khi chưa login xong.")
            exit()

    # ✅ Lấy Student ID
    student_id = get_student_id_from_profile(driver)
    if not student_id:
        exit()

    # ✅ Truy cập trang đơn từ
    print("📄 Đang vào trang Đơn từ...")
    driver.get(ACAD_URL)
    while True:
        try:
            if "AcadAppView.aspx" in driver.current_url:
                break
            driver.title
            time.sleep(1)
        except:
            print("❌ Trình duyệt bị đóng.")
            exit()

    time.sleep(2)
    html = driver.page_source
    df = extract_acad_data_from_page_source(html, student_id)
    write_new_csv(df, CSV_FILE)


🌐 Mở FAP và chờ bạn đăng nhập bằng tay...
🎓 Mã số sinh viên: DE190709
📄 Đang vào trang Đơn từ...
✅ Đã lưu dữ liệu 15 dòng vào: don_tu.csv


In [2]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from seleniumbase import SB

class FAPScraper:
    def __init__(self, profile_url="https://fap.fpt.edu.vn/User/Profile.aspx",
                 acad_url="https://fap.fpt.edu.vn/App/AcadAppView.aspx",
                 csv_file="don_tu.csv"):
        self.profile_url = profile_url
        self.acad_url = acad_url
        self.csv_file = csv_file
        self.student_id = None

    def _wait_for_page(self, driver, target_url, page_name):
        """Đợi cho đến khi trang được tải xong."""
        while True:
            try:
                if target_url in driver.current_url:
                    print(f"✅ Đã vào trang {page_name}.")
                    return True
                driver.title
                time.sleep(1)
            except:
                print(f"❌ Trình duyệt đã đóng khi vào trang {page_name}.")
                return False

    def login(self, driver):
        """Đăng nhập vào FAP."""
        print("🌐 Mở FAP và chờ bạn đăng nhập bằng tay...")
        driver.get("https://fap.fpt.edu.vn")
        return self._wait_for_page(driver, "Student.aspx", "đăng nhập")

    def get_student_id(self, driver):
        """Lấy mã số sinh viên từ trang Profile."""
        driver.get(self.profile_url)
        if not self._wait_for_page(driver, "Profile.aspx", "Profile"):
            return None

        time.sleep(1)
        try:
            student_id = driver.find_element("id", "ctl00_mainContent_lblRollNumber").text.strip()
            print(f"🎓 Mã số sinh viên: {student_id}")
            self.student_id = student_id
            return student_id
        except Exception as e:
            print(f"❌ Không lấy được Student ID: {e}")
            return None

    def extract_acad_data(self, driver):
        """Trích xuất dữ liệu từ trang AcadAppView."""
        driver.get(self.acad_url)
        if not self._wait_for_page(driver, "AcadAppView.aspx", "Đơn từ"):
            return None

        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        table = soup.find("div", id="ctl00_mainContent_content").find("table")
        
        if not table:
            print("❌ Không tìm thấy bảng dữ liệu.")
            return pd.DataFrame()

        rows = table.find_all("tr")[1:]  # Bỏ dòng tiêu đề
        data = []
        for row in rows:
            cols = row.find_all("td")
            if len(cols) >= 7:
                record = {
                    "StudentID": self.student_id,
                    "Type": cols[0].text.strip(),
                    "Purpose": cols[1].text.strip(),
                    "CreateDate": cols[2].text.strip(),
                    "ProcessNote": cols[3].text.strip(),
                    "File": cols[4].text.strip(),
                    "Status": cols[5].text.strip(),
                    "Timestamp": cols[6].text.strip()
                }
                data.append(record)

        return pd.DataFrame(data)

    def save_to_csv(self, df):
        """Lưu dữ liệu vào file CSV."""
        if df.empty:
            print("⚠️ Không có dữ liệu để lưu.")
            return
        df.to_csv(self.csv_file, index=False, encoding="utf-8-sig")
        print(f"✅ Đã lưu dữ liệu {len(df)} dòng vào: {self.csv_file}")

    def run(self):
        """Chạy toàn bộ quy trình."""
        with SB(uc=True, headless=False) as sb:
            driver = sb.driver
            driver.set_window_size(1920, 1080)
            try:
                if not self.login(driver):
                    print("❌ Đăng nhập thất bại.")
                    return

                student_id = self.get_student_id(driver)
                if not student_id:
                    print("❌ Không thể tiếp tục do không lấy được Student ID.")
                    return

                df = self.extract_acad_data(driver)
                self.save_to_csv(df)
            except Exception as e:
                print(f"❌ Lỗi trong quá trình chạy: {e}")
            finally:
                print("🛑 Đã đóng trình duyệt.")

# Thực thi
if __name__ == "__main__":
    scraper = FAPScraper()
    scraper.run()

🌐 Mở FAP và chờ bạn đăng nhập bằng tay...
✅ Đã vào trang đăng nhập.
✅ Đã vào trang Profile.
🎓 Mã số sinh viên: DE190709
✅ Đã vào trang Đơn từ.
✅ Đã lưu dữ liệu 15 dòng vào: don_tu.csv
🛑 Đã đóng trình duyệt.
